In [23]:
import numpy as np
import pandas as pd
import xgboost as xgb
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

data_path = Path().cwd() / 'data'

In [24]:
test_sheet = 'test_points_extracted.xls'
test_sites = pd.read_excel(data_path / test_sheet, engine='calamine')

test_sites.drop(columns=[col for col in test_sites.columns if 'NEAR_FID' in col], inplace=True)

print(test_sites.shape)
test_sites.head()

(12067, 12)


,OBJECTID_1,OBJECTID,x,y,NEAR_DIST_Canals,NEAR_DIST_River_Net,NEAR_DIST_Coastal,NEAR_DIST_Chert,Elevation,Wetness,Temp,Slope
0,1,3353,15.726739,43.023390,325051.049972,92375.044496,109.719805,27245.870159,70.596275,0,16.799999,207
1,2,3357,15.743060,43.023390,326171.018246,91240.568066,521.790859,25924.985926,227.165344,0,16.400000,243
2,3,3362,15.759382,43.023390,327292.421342,90111.311563,128.050137,24605.088261,41.271420,0,17.100000,236
3,4,3410,15.759382,43.032675,326739.480379,89566.947218,238.350808,24494.102940,58.482124,0,16.500000,232
4,5,183,16.510161,42.763406,376552.349498,77429.587911,45.335448,792.107078,22.511099,0,17.000000,242


In [25]:
def clean_waw(df: pd.DataFrame):
    wcols = [col for col in df.columns if 'WAW' in col]
    waw = df[wcols + ['OBJECTID']]
    df.drop(columns=wcols, inplace=True)
    
    waw['Wetness'] = waw[waw.drop(['OBJECTID'], axis=1).columns].apply(
        lambda x: ''.join(x.dropna().astype(str)), 
        axis=1
    )
    
    waw = waw[waw['Wetness'] != ''][['OBJECTID', 'Wetness']]
    waw['Wetness'] = waw['Wetness'].astype(float).astype(np.uint8)
    return pd.merge(df, waw, on='OBJECTID', how='inner')

In [26]:
if any('WAW' in col for col in test_sites.columns):
    test_cleaned = clean_waw(test_sites)

else:
    test_cleaned = test_sites.copy()
    
test_cleaned.dropna(inplace=True)

test_cleaned.to_csv(data_path / f"{test_sheet.split('.')[0]}_cleaned.csv", index=False)

In [27]:
known_sites = pd.read_excel(data_path / 'known_sites_augmented.xls', engine='calamine')

known_sites.rename(columns={
        i: i.replace('sites_XYTableToPoint_', '') for i in known_sites.columns if 'sites_XYTableToPoint_' in i
    }, inplace=True)

known_sites.rename(columns={
    'Elevation__Masl_': 'Elevation',
    'Dd_ns': 'y',
    'Dd_ew': 'x',
    }, inplace=True)

known_sites.drop(columns=['Dd', 'Dms'] + [col for col in known_sites.columns if 'NEAR_FID' in col], inplace=True)

print(known_sites.shape)
known_sites.head()

(47, 39)


,OBJECTID,Site_Name,Geographical_Region,Geographical_Location,Elevation,Period_New,Site_Type,y,x,NEAR_DIST_Chert,...,WAW_2018_010m_E49N21_03035_v020,WAW_2018_010m_E49N22_03035_v020,WAW_2018_010m_E49N24_03035_v020,WAW_2018_010m_E49N25_03035_v020,WAW_2018_010m_E50N21_03035_v020,WAW_2018_010m_E50N22_03035_v020,WAW_2018_010m_E50N24_03035_v020,WAW_2018_010m_E50N25_03035_v020,Slope,Temp
0,1,Abri Kontija 002,Istra,Limski kanal,46.0,UP,RS,45.137500,13.718611,247901.167568,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,204,15.7
1,2,Abri Šebrn,NaN,NaN,750.0,MES,RS,45.337712,14.162687,235748.803581,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,241,11.0
2,3,Brjgućeva Loza 1 (Loza),Istra,Kastav,510.0,MES,C,45.467778,14.242222,241293.250685,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,247,12.4
3,4,Bukovac,NaN,NaN,864.0,UP,C,45.346569,14.756238,204549.089663,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,245,9.7
4,5,Campanož,NaN,NaN,NaN,MP,O,44.849045,13.899990,215799.109406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250,15.7


In [28]:
known_sites_cleaned = clean_waw(known_sites).drop(columns=['Geographical_Region', 'Geographical_Location', 'Period_New', 'Site_Type'])

cols = known_sites_cleaned.columns
known_sites_cleaned = known_sites_cleaned[cols[:3].to_list() + [cols[-2]] + cols[3:-2].to_list() + [cols[-1]]]

known_sites_cleaned.Elevation.fillna(known_sites_cleaned.Elevation_Raster, inplace=True)
known_sites_cleaned.Elevation.fillna(0, inplace=True)

known_sites_cleaned.drop(columns=['Elevation_Raster'], inplace=True)

known_sites_cleaned['Is_Site'] = 1

# Modeling

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [30]:
print(f'known columns: {known_sites_cleaned.columns}')
print(f'test columns: {test_cleaned.columns}')

known columns: Index(['OBJECTID', 'Site_Name', 'Elevation', 'Temp', 'y', 'x',
       'NEAR_DIST_Chert', 'NEAR_DIST_Canals', 'NEAR_DIST_River_Net',
       'NEAR_DIST_Coastal', 'Slope', 'Wetness', 'Is_Site'],
      dtype='object')
test columns: Index(['OBJECTID_1', 'OBJECTID', 'x', 'y', 'NEAR_DIST_Canals',
       'NEAR_DIST_River_Net', 'NEAR_DIST_Coastal', 'NEAR_DIST_Chert',
       'Elevation', 'Wetness', 'Temp', 'Slope'],
      dtype='object')


In [31]:
known = known_sites_cleaned.drop(columns=['OBJECTID', 'x', 'y', 'Site_Name'])
test = test_cleaned.drop(columns=['OBJECTID', 'x', 'y'])

col_order = ['Elevation', 'Wetness', 'Temp', 'Slope', 'NEAR_DIST_Chert', 'NEAR_DIST_Canals', 'NEAR_DIST_River_Net', 'NEAR_DIST_Coastal']

known = known[col_order + ['Is_Site']]
test = test[col_order]

test.reset_index(drop=True, inplace=True)

In [32]:
known.head()

,Elevation,Wetness,Temp,Slope,NEAR_DIST_Chert,NEAR_DIST_Canals,NEAR_DIST_River_Net,NEAR_DIST_Coastal,Is_Site
0,46.00000,0,15.7,204,247901.167568,80993.524258,73640.087050,7612.808936,1
1,750.00000,0,11.0,241,235748.803581,98999.561639,44437.690620,9921.595682,1
2,510.00000,0,12.4,247,241293.250685,101016.428031,34876.914519,14119.067154,1
3,864.00000,0,9.7,245,204549.089663,143240.912311,13416.969148,15491.977574,1
4,43.70879,0,15.7,250,215799.109406,114332.595817,94426.986888,2230.743465,1


In [33]:
test.head()

,Elevation,Wetness,Temp,Slope,NEAR_DIST_Chert,NEAR_DIST_Canals,NEAR_DIST_River_Net,NEAR_DIST_Coastal
0,70.596275,0,16.799999,207,27245.870159,325051.049972,92375.044496,109.719805
1,227.165344,0,16.400000,243,25924.985926,326171.018246,91240.568066,521.790859
2,41.271420,0,17.100000,236,24605.088261,327292.421342,90111.311563,128.050137
3,58.482124,0,16.500000,232,24494.102940,326739.480379,89566.947218,238.350808
4,22.511099,0,17.000000,242,792.107078,376552.349498,77429.587911,45.335448


In [34]:
test.loc[(test['Elevation'] > 400), 'Is_Site'] = 0
test.loc[(test['Elevation'] <= 400), 'Is_Site'] = 1
# test.loc[(test['Elevation'] < 50), 'Is_Site'] = 0


test.loc[(test['NEAR_DIST_Chert'] > 20000), 'Is_Site'] = 0
test.loc[(test['NEAR_DIST_Coastal'] > 6000), 'Is_Site'] = 0
# test.loc[(test['NEAR_DIST_Canals'] < 20000), 'Is_Site'] = 0
# test.loc[(test['NEAR_DIST_River_Net'] < 20000), 'Is_Site'] = 0


test['Is_Site'] = test['Is_Site'].astype(np.uint8)

test.to_csv(data_path / 'test_labeled.csv', index=False)

test['Is_Site'].value_counts()

Is_Site
0    10769
1     1297
Name: count, dtype: int64

In [35]:
rng = np.random.RandomState()

RNG_MIX = 0.01

rng_selection = test[test['Is_Site'] == 0]
rng_nums = rng.choice(
    a=rng_selection.index, 
    size=int(test['Is_Site'].value_counts()[0]*RNG_MIX), 
    replace=False
)

test_cut = test.drop(index=rng_nums) # pyright: ignore[reportArgumentType, reportCallIssue]

rng_sites = rng_selection.loc[rng_nums]

rng_sites

,Elevation,Wetness,Temp,Slope,NEAR_DIST_Chert,NEAR_DIST_Canals,NEAR_DIST_River_Net,NEAR_DIST_Coastal,Is_Site
2666,273.824158,0,14.900000,247,19833.003844,282798.092084,29274.721754,21169.322419,0
5473,1070.443115,0,9.800000,249,5983.270933,272200.780968,20717.477132,34323.930662,0
511,233.356323,0,15.800000,250,29440.869702,312687.742838,8893.552488,4720.101456,0
11019,1266.715332,0,7.900000,227,129272.068314,200910.021327,13370.053601,11556.930321,0
8994,603.984680,0,11.400000,249,80172.802628,217711.667734,22920.477053,20481.299817,0
...,...,...,...,...,...,...,...,...,...
10920,1018.554138,0,10.000000,231,126205.079655,203559.857992,18732.592396,3517.813489,0
5913,251.926620,0,15.300000,250,9037.736412,263433.880200,18327.325836,36933.429573,0
7152,897.890442,0,10.800000,231,16721.614219,264972.084237,17819.055430,43038.398781,0
9813,1089.098389,0,9.800000,245,21953.188560,246592.684047,27613.635734,53994.799046,0


In [36]:
known_rng = pd.concat([known, rng_sites], ignore_index=True)

print(f'Using a combination of {known.shape[0]} known sites and {rng_sites.shape[0]} test sites for {known_rng.shape[0]} total test sites.')

known_rng

Using a combination of 47 known sites and 107 test sites for 154 total test sites.


,Elevation,Wetness,Temp,Slope,NEAR_DIST_Chert,NEAR_DIST_Canals,NEAR_DIST_River_Net,NEAR_DIST_Coastal,Is_Site
0,46.000000,0,15.700000,204,247901.167568,80993.524258,73640.087050,7612.808936,1
1,750.000000,0,11.000000,241,235748.803581,98999.561639,44437.690620,9921.595682,1
2,510.000000,0,12.400000,247,241293.250685,101016.428031,34876.914519,14119.067154,1
3,864.000000,0,9.700000,245,204549.089663,143240.912311,13416.969148,15491.977574,1
4,43.708790,0,15.700000,250,215799.109406,114332.595817,94426.986888,2230.743465,1
...,...,...,...,...,...,...,...,...,...
149,1018.554138,0,10.000000,231,126205.079655,203559.857992,18732.592396,3517.813489,0
150,251.926620,0,15.300000,250,9037.736412,263433.880200,18327.325836,36933.429573,0
151,897.890442,0,10.800000,231,16721.614219,264972.084237,17819.055430,43038.398781,0
152,1089.098389,0,9.800000,245,21953.188560,246592.684047,27613.635734,53994.799046,0


In [37]:
X_train, X_test, y_train, y_test = train_test_split(known_rng.drop(columns=['Is_Site']), known_rng['Is_Site'], test_size=0.25, stratify=known_rng['Is_Site'])

In [38]:
model = xgb.XGBClassifier(tree_method='hist')

model.fit(X_train, y_train, eval_set=[(X_test, y_test)])

y_pred = model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred.round())*100}%')

[0]	validation_0-logloss:0.51054
[1]	validation_0-logloss:0.45924
[2]	validation_0-logloss:0.42576
[3]	validation_0-logloss:0.36756
[4]	validation_0-logloss:0.35347
[5]	validation_0-logloss:0.32472
[6]	validation_0-logloss:0.30706
[7]	validation_0-logloss:0.29604
[8]	validation_0-logloss:0.27587
[9]	validation_0-logloss:0.26971
[10]	validation_0-logloss:0.26255
[11]	validation_0-logloss:0.25688
[12]	validation_0-logloss:0.25636
[13]	validation_0-logloss:0.25961
[14]	validation_0-logloss:0.25338
[15]	validation_0-logloss:0.25930
[16]	validation_0-logloss:0.25842
[17]	validation_0-logloss:0.25571
[18]	validation_0-logloss:0.25653
[19]	validation_0-logloss:0.25507
[20]	validation_0-logloss:0.25465
[21]	validation_0-logloss:0.25554
[22]	validation_0-logloss:0.25437
[23]	validation_0-logloss:0.25767
[24]	validation_0-logloss:0.26175
[25]	validation_0-logloss:0.26147
[26]	validation_0-logloss:0.26054
[27]	validation_0-logloss:0.25679
[28]	validation_0-logloss:0.25875
[29]	validation_0-loglos

In [39]:
outcome = test_cut.drop(columns=['Is_Site'])
outcome['prediction'] = model.predict(outcome)

outcome

,Elevation,Wetness,Temp,Slope,NEAR_DIST_Chert,NEAR_DIST_Canals,NEAR_DIST_River_Net,NEAR_DIST_Coastal,prediction
0,70.596275,0,16.799999,207,27245.870159,325051.049972,92375.044496,109.719805,1
1,227.165344,0,16.400000,243,25924.985926,326171.018246,91240.568066,521.790859,1
2,41.271420,0,17.100000,236,24605.088261,327292.421342,90111.311563,128.050137,1
3,58.482124,0,16.500000,232,24494.102940,326739.480379,89566.947218,238.350808,1
4,22.511099,0,17.000000,242,792.107078,376552.349498,77429.587911,45.335448,1
...,...,...,...,...,...,...,...,...,...
12061,551.129211,0,10.300000,250,90956.384346,184989.356662,2271.270992,59132.748107,0
12062,659.387390,0,9.800000,222,91942.567948,183977.412603,3259.110714,59923.992081,0
12063,337.125427,0,11.300000,216,88604.594675,186768.663311,41.657071,58440.290816,0
12064,400.164185,0,10.900000,236,89592.956286,185754.992778,1064.431584,59204.693096,0


In [40]:
outcome['prediction'].value_counts()

prediction
0    9774
1    2185
Name: count, dtype: int64

In [41]:
outcome_linked = outcome.merge(test_cleaned[['OBJECTID', 'x', 'y']], left_index=True, right_index=True)
outcome_linked

,Elevation,Wetness,Temp,Slope,NEAR_DIST_Chert,NEAR_DIST_Canals,NEAR_DIST_River_Net,NEAR_DIST_Coastal,prediction,OBJECTID,x,y
0,70.596275,0,16.799999,207,27245.870159,325051.049972,92375.044496,109.719805,1,3353,15.726739,43.023390
1,227.165344,0,16.400000,243,25924.985926,326171.018246,91240.568066,521.790859,1,3357,15.743060,43.023390
2,41.271420,0,17.100000,236,24605.088261,327292.421342,90111.311563,128.050137,1,3362,15.759382,43.023390
3,58.482124,0,16.500000,232,24494.102940,326739.480379,89566.947218,238.350808,1,3410,15.759382,43.032675
4,22.511099,0,17.000000,242,792.107078,376552.349498,77429.587911,45.335448,1,183,16.510161,42.763406
...,...,...,...,...,...,...,...,...,...,...,...,...
12061,551.129211,0,10.300000,250,90956.384346,184989.356662,2271.270992,59132.748107,0,25454,16.004201,44.666859
12062,659.387390,0,9.800000,222,91942.567948,183977.412603,3259.110714,59923.992081,0,25455,16.004201,44.676145
12063,337.125427,0,11.300000,216,88604.594675,186768.663311,41.657071,58440.290816,0,25456,16.004201,44.685430
12064,400.164185,0,10.900000,236,89592.956286,185754.992778,1064.431584,59204.693096,0,25457,16.020522,44.657574


In [42]:
outcome_linked[(outcome_linked['y'] <= 43.57) & (outcome_linked['prediction'] == 1)].shape

(1071, 12)

In [43]:
%%script True
outcome_linked.to_csv(data_path / f'test_predictions_{RNG_MIX}.csv', index=False)
outcome_linked[outcome_linked['prediction'] == 1].to_csv(data_path / f'test_predictions_sites_{RNG_MIX}.csv', index=False)

Couldn't find program: 'True'
